In [36]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import cv2

import pickle

In [15]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import imageio.v3 as iio
import os
from datetime import timedelta
import sys
from pathlib import Path 

## Подготовка данных из видео

Предполагается, что видео для инференса подгружаются в папку data/test/videos

### Служебные функции

In [16]:
def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")

In [37]:
def main(video_name, model_file, delta):
    """
    Функция выделяет в видео отрезки с нарушенями и за __ секунд до нарушения,
    создает кадры и сохраняет их в заданную папку.
    
    Args:
        video_name - название видео
        delta - сколько секунд отступаем до нарушения
        model_file - имя файла с моделью
    """
    # получить FPS видео
    metadata = iio.immeta(os.path.join(os.getcwd(), 'data', 'test', 'videos', video_name),exclude_applied=False)
    fps = metadata['fps']
    # если SAVING_FRAMES_PER_SECOND выше видео FPS, то установите его на FPS (как максимум)
    # получить список длительностей для сохранения
    # запускаем цикл
    count = 0
    save_frame_count = 0
    res = {'frametime': [],
            'score': []}
    model = pickle.load(open(os.path.join(os.getcwd(), model_file), 'rb'))
    for frame in iio.imiter(os.path.join(os.getcwd(), 'data', 'test', 'videos', video_name), plugin="pyav"):
        res['frametime'].append(count / fps)
        if model_file == 'baseline.pkl':
            res['score'].append(model.predict(cv2.resize(frame,(224, 224))))
        elif model_file == 'ur_model_filename':
            pass # добавить свою логику
        else:
            print('ПРОПИШИ ЛОГИКУ ПОД СВОЙ ПИКЛ С МОДЕЛЬЮ В СКРИПТЕ')
            break
        count += 1
    res = pd.DataFrame(res)
    res['model'] = model_file
    res['filename'] = video_name
    return res

## Инференс

In [38]:
SAVING_FRAMES_PER_SECOND = 1 # кол-во кадров  в секунду
DELTA = 20 # отступаем до начала нарушения 5 сек
MODEL_FILE = 'baseline.pkl'

In [39]:
save_result_path = os.path.join(os.getcwd(), MODEL_FILE, 'scores.csv')


inf_result = pd.DataFrame()

for f in tqdm([
x for x in os.listdir(os.path.join(os.getcwd(), 'data', 'test', 'videos')) if x.lower().endswith('.mp4')
]):
    video_file = os.path.join(os.getcwd(), 'data', 'test', 'videos', f'{f}')  
    video_df = main(f, MODEL_FILE, DELTA)   
    inf_result = pd.concat([image_result_info, video_df], ignore_index=True)
inf_result.to_csv(save_result_path)

  0%|          | 0/1 [00:26<?, ?it/s]


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "seq